In [55]:
## Agrupacion y Segmentacion de Vecindarios de Toronto 

#### Parte 1

In [56]:
# Importacion de Librerias necesarias

In [57]:
pip install BeautifulSoup4

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [58]:
pip install lxml

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [59]:
from bs4 import BeautifulSoup  
import requests 
import pandas as pd
import lxml


In [60]:
#Importacion , Scraping y Creacion de Dataframe

In [61]:
url =requests.get ("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=958430791").text

In [62]:
soup = BeautifulSoup(url,'lxml')

In [63]:
My_table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(My_table))[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [64]:
# Eliminamos Municipios 'Not assigned'

In [65]:
drop_Not=df[df['Borough']=='Not assigned'].index
df.drop (drop_Not, inplace=True)
df.reset_index(drop=True, inplace=True)

In [66]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [67]:
# Asignamos Vecindarios 'Not assigned' a Municipio

In [68]:
df[df['Neighborhood']=='Not assigned']=df['Borough'].reset_index()
df.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [69]:
# Requerimiento filas-columnas

In [70]:
df.shape

(103, 3)

#### Parte 2

In [71]:
# Importacion de archivo csv con coordenadas

In [72]:

from io import StringIO

url = requests.get(' http://cocl.us/Geospatial_data')
csv_raw = StringIO(url.text)
datos_geo = pd.read_csv(csv_raw)

In [73]:
datos_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [74]:
# Relacionamos Dataframes 

In [75]:
df=pd.merge(df,datos_geo,how='left')


In [76]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Parte 3

In [77]:
# Importacion Librerias Necesarias

In [78]:
import numpy as np
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
print('Librerias importadas')

Solving environment: done

# All requested packages already installed.

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Librerias importadas


In [79]:
# Filtramos a Municipios que sean ' Toronto '


Tor_d = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Tor_d.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [80]:
# Obtenemos nro filas 

Tor_d.shape

(39, 5)

In [81]:
# Obtenemos coordenadas para Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordenadas Toronto {}, {}.'.format(latitude, longitude))

Coordenadas Toronto 43.6534817, -79.3839347.


In [82]:
# Creamos mapa de Toronto y añadimos los marcadores

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Tor_d['Latitude'], Tor_d['Longitude'], Tor_d['Borough'], Tor_d['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=2,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [83]:
# Definimos version y credenciales para la API de Foursquare con un limite de 50 devoluciones

CLIENT_ID = 'DBBPKNBZNEC2JFO4JL13AJOJLMHLHOCOYWGTY5ZXJFXRJXYD' 
CLIENT_SECRET = 'YA51Q1Y4XHBRUZE1025SHVN3AKMX20MJHBBELS0BANSJWKMA' 
VERSION = '20180605' 
LIMIT = 50 


In [84]:
# Definimos funcion para obtener los 100 sitios en un radio de 500 mts para cada barrio

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
                # solicitud GET
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
# Corremos la funcion getNearbyVenues para cada barrio creando un Dataframe 'Toronto_venues'

Toronto_venues = getNearbyVenues(names=Tor_d['Neighborhood'],
                                   latitudes=Tor_d['Latitude'],
                                   longitudes=Tor_d['Longitude'])
                                  

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [86]:
# mostramos Dataframe y revisamos tamaño

print(Toronto_venues.shape)
Toronto_venues.head()

(1172, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [87]:
# Observemos cuantos sitios se devolvieron por barrio

Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,50,50,50,50,50,50
Christie,15,15,15,15,15,15
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
Davisville,35,35,35,35,35,35


In [88]:
# Vemos cuantas categorias unicas hay por barrio

print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 217 uniques categories.


In [89]:
#Analizamos cada barrio



# codificación
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.shape

(1172, 217)

In [90]:
# Agrupamos las filas por barrios tomando el parametro de promedio de ocurrencia de cada categoria


Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.00000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000
2,"Business reply mail Processing Centre, South C...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00,0.071429,0.071429,0.142857,0.071429,0.142857,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000
4,Central Bay Street,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.00000
5,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000
6,Church and Wellesley,0.040000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.02,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000
7,"Commerce Court, Victoria Hotel",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.00000
8,Davisville,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.057143,0.00000,0.00,0.000000,0.000000,0.000000,0.00000
9,Davisville North,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00000


In [91]:
#Observamos tamaño Dataframe

Toronto_grouped.shape

(39, 217)

In [92]:
# Visualizamos cada barrio con los 12 lugares mas comunes

num_top_venues = 12

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                  venue  freq
0           Coffee Shop  0.06
1              Beer Bar  0.04
2            Restaurant  0.04
3                Bakery  0.04
4          Cocktail Bar  0.04
5        Farmers Market  0.04
6    Seafood Restaurant  0.04
7           Cheese Shop  0.04
8   Moroccan Restaurant  0.02
9                 Hotel  0.02
10           Steakhouse  0.02
11               Bistro  0.02


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.12
1             Coffee Shop  0.08
2          Breakfast Spot  0.08
3                  Bakery  0.08
4           Grocery Store  0.04
5      Italian Restaurant  0.04
6       Convenience Store  0.04
7   Performing Arts Venue  0.04
8                     Gym  0.04
9               Nightclub  0.04
10             Restaurant  0.04
11           Climbing Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                   venue

In [93]:
# Creamos una funcion para ordenar en forma descendente los lugares

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
# mostramos los primeros 12 lugares de cada barrio en un Dataframe nuevo

num_top_venues = 12

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Farmers Market,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant,Clothing Store,Pub,Bistro,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Gym / Fitness Center,Gym,Climbing Gym,Restaurant,Intersection,Stadium,Pet Store,Grocery Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Gym / Fitness Center,Pizza Place,Restaurant,Auto Workshop,Spa,Burrito Place,Fast Food Restaurant,Farmers Market,Comic Shop,Skate Park,Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Terminal,Boat or Ferry,Plane,Rental Car Location,Airport,Airport Food Court,Airport Service,Boutique,Sculpture Garden,Coffee Shop,Harbor / Marina
4,Central Bay Street,Coffee Shop,Sandwich Place,Bubble Tea Shop,Café,Italian Restaurant,Comic Shop,Salad Place,Burger Joint,Portuguese Restaurant,Poke Place,Park,Office


In [95]:
# establecer el número de agrupaciones en 5
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:5] 


array([0, 0, 0, 0, 0], dtype=int32)

In [96]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Tor_d

# juntar Toronto_grouped con Toronto_data 
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].fillna("0").astype(int)


Toronto_merged.head() # revisar las ultimas columnas

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Restaurant,Cosmetics Shop,Distribution Center,Dessert Shop,Electronics Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Spa,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium,College Cafeteria
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Café,Bookstore,Clothing Store,Ramen Restaurant,Theater,Cosmetics Shop,Fast Food Restaurant,Burrito Place,Hotel,Diner,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Cosmetics Shop,Gastropub,Coffee Shop,Seafood Restaurant,Bakery,Farmers Market,Park,Japanese Restaurant,Creperie,Bookstore,Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Wine Bar,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [97]:
# creamos mapa de agrupacion
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
# Examinamos las agrupaciones pudiendo observar que categorias distinguen a cada agrupacion

In [99]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Restaurant,Cosmetics Shop,Distribution Center,Dessert Shop,Electronics Store
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Spa,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café,College Auditorium,College Cafeteria
2,"Garden District, Ryerson",0,Coffee Shop,Café,Bookstore,Clothing Store,Ramen Restaurant,Theater,Cosmetics Shop,Fast Food Restaurant,Burrito Place,Hotel,Diner,Restaurant
3,St. James Town,0,Café,Cosmetics Shop,Gastropub,Coffee Shop,Seafood Restaurant,Bakery,Farmers Market,Park,Japanese Restaurant,Creperie,Bookstore,Restaurant
4,The Beaches,0,Trail,Pub,Health Food Store,Wine Bar,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
5,Berczy Park,0,Coffee Shop,Beer Bar,Farmers Market,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Restaurant,Clothing Store,Pub,Bistro,Concert Hall
6,Central Bay Street,0,Coffee Shop,Sandwich Place,Bubble Tea Shop,Café,Italian Restaurant,Comic Shop,Salad Place,Burger Joint,Portuguese Restaurant,Poke Place,Park,Office
7,Christie,0,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Nightclub,Restaurant,Coffee Shop,Baby Store,Department Store,Dessert Shop,Distribution Center
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Concert Hall,Steakhouse,American Restaurant,Thai Restaurant,Pizza Place,Restaurant,Noodle House,Japanese Restaurant,Seafood Restaurant,Salon / Barbershop
9,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Park,Supermarket,Pet Store,Middle Eastern Restaurant,Portuguese Restaurant,Café,Brewery,Bar,Bank,Music Venue


In [100]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
18,Lawrence Park,1,Park,Bus Line,Swim School,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [101]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
29,"Moore Park, Summerhill East",2,Park,Trail,Playground,Restaurant,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
33,Rosedale,2,Park,Trail,Playground,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [102]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",3,Park,Jewelry Store,Trail,Sushi Restaurant,Wine Bar,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [103]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
19,Roselawn,4,Garden,Music Venue,Health & Beauty Service,Wine Bar,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
